# Stroop Task Sequence

Here, we want to create a balanced sequence for the Stroop task. In a Stroop task, an inked color word is shown. Here, we use four different stimuli: <span style="color:red">RED</span>, <span style="color:blue">BLUE</span>, <span style="color:red">RED</span>, <span style="color:blue">BLUE</span>. The task of the participant is to name the color of the word as fast as possible while ignoring the meaning.

In the Stroop task, stimuli where the color and the meaning of the word match (<span style="color:red">RED</span>, <span style="color:blue">BLUE</span>) are called congruent stimuli. Stimuli where the color and meaning mismatch (<span style="color:blue">RED</span>, <span style="color:red">BLUE</span>) are called incongruent. It is an extremely robust psychological finding that congruent stimuli lead to faster and more accurate responses than incongruent stimuli. This phenomenon is called `congruency effect`.

## Counterbalance schema

Here, we want to replicate the findings and balance multiple stimuli features. In addition to the color, meaning and congruency, we also know that response transitions might have an influence on how well the task is performed. For example, the sequence

(1) <span style="color:blue">BLUE</span>, <span style="color:blue">RED</span>, <span style="color:blue">RED</span>, <span style="color:blue">RED</span>, <span style="color:red">BLUE</span>, <span style="color:red">RED</span>, <span style="color:red">BLUE</span>, <span style="color:red">BLUE</span>

might be easier then

(2) <span style="color:blue">BLUE</span>, <span style="color:red">RED</span>, <span style="color:red">BLUE</span>, <span style="color:blue">BLUE</span>, <span style="color:blue">RED</span>, <span style="color:red">RED</span>, <span style="color:blue">BLUE</span>, <span style="color:red">RED</span>

despite the fact, that the first sequence contains more incongruent stimuli. This is due the fact that there is only one response switch in the first sequence, but multiple response switches in the second list.

### Factors to balance

Here, we want to balance the following factors. To introduce the sweetPea terminology, we call features that we want to balance "factors". The individual characteristics in which they can occur are called "levels".

- *Color*: same amount of `red` and `blue` words
- *Word*: same amount of `'RED'` and `'BLUE'` as word stimulus
- *Congruency*: same amount of `congruent` and `incongruent` trials
- *Response transitions*: same amount of `response switches` as `response repetitions`

## Crossing
Factors can be balanced individually or **crossed**. In many cases, we not only want to balance factors individually, but cross them:

In our case we not only want the same amount of `congruent` and `incongruent` tasks, but we also want the same amount of `blue`, `congruent` tasks as `blue`, `incongruent` tasks as `red`, `congruent` tasks as `red`, `incongruent` tasks. In other words, we want to cross the *color* and the *congruency* factor.

The same is true for the *response transitions*. We want the same amount of `blue`, `response switch` tasks as `blue`, `response repetition` tasks as `red`, `response switch` tasks as `red`, `response repetition` tasks. In other words, we want to cross the *color* and the *response transition* factor.

<br>

**!!! WARNING !!!**

*Crossings of certain factors might lead to automatically counterbalancing other factors.*

*Some crossings are impossible.*

In our use case, crossing the *color* of a word and its *word* automatically generates balanced *congruency* (this is only the case when using exactly **two** color levels and **two** different word levels).

On the other hand, crossing *color*, *word* and *congruency* all together is impossible. There is no stimulus that is inked `red`, that's word is `'RED'` and that is `incongruent`. In other words, there is no sensible crossing between *color*, *word* and *congruency* since some level combinations are impossible. This is often the case when dealing with "derived factors". "Derived factors" are factors that depend on other factors. For example, the *congruency* depends on the stimulus *color* and its *word* and solely derives from these factors.

*-> Although sweetPea is an amazing tool to create counterbalanced sequences and cross multiple factors including transition factors, we still need to carefully think about our counterbalancing schema before starting to code.*

## Creating the sequence with sweetPea
### Installing SweetPea

First, we need to install sweetPea. Here, we use `pip` to install our package.

*Note*
In jupiter notebooks we can use an `!` in the beginning of the cell to call shell commands. If you are writing a python script, these commands would not be part of your script but be used in your terminal  (without the `!`).

Also, %% is `cell magic` in jupiter notebooks. In this case, we supress the output of the cell.

In [39]:
%%capture

!pip install sweetpea

### Importing

Here, we import functionality from sweetPea. We explain the different functions and classes in more detail later.

In [40]:
from sweetpea import (Factor, DerivedLevel, WithinTrial, Transition, CrossBlock, MinimumTrials, synthesize_trials, CMSGen, print_experiments, experiments_to_dicts)

### Regular Factors

In this experiment, there are two regular factors: *color* and *word*. Both of these factors have two Levels. First, we declare these factors. As arguments, we give it a name and the levels.

In [41]:
# Declare the color factor
color = Factor(name='color', initial_levels=['red', 'blue'])

# Declare the word factor
word = Factor(name='word', initial_levels=['RED', 'BLUE'])

### Derived Factor: Congruency

Here, we declare *congruency* as derived factor. For reasons mentioned earlier, we will not include this factor in the counterbalancing schema, but we still include it to demonstrate the concept. Also, it will show up in the generated trial sequence, and we can use it as datapoint in the analysis.

#### Defining functions for each congruency level
First, we need to declare functions that determine weather a trial is `congruent` or `incongruent`. For each level of congruency, we need a function.

In [42]:
# A trial is congruent if the color matches the lowercase version of its word
def is_congruent(col, wrd):
    return col == wrd.lower()

# A trial is incongruent if it is not congruent
def is_incongruent(col, wrd):
    return not is_congruent(col, wrd)

*Note*
The above code might be confusing and written to be short. If in doubt, one can always nest if else statements like so:

```python
def is_congruent(col, wrd):
    if col == 'red' and wrd == 'RED':
        return True
    elif col == 'red' and wrd == 'BLUE':
        return False
    elif col == 'blue' and wrd == 'RED':
        return False
    elif col == 'blue' and wrd == 'BLUE':
        return True

def is_incongruent(col, wrd):
    ...
```

However, defining is_incongruent as negation of is_congruent is good practice. It ensures, that a stimulus can either be `congruent` or `incongruent` and there is no undefined factor level for *congruency*. We use the "WithinTrial" method to indicate that congruency depends on other factors within one trial (later we will show how to deal with transitions that depend on factors of different trials, e.g. of the current and a previous trial)

#### Defining the derived levels
Now, we declare the levels with the use of the functions. A derived level expects a name and a "Window". The "Window" itself needs a function to call and the trial factors (as a list) on witch to call the function. For example, the `congruent` level is named "congruent" and uses a WithinTrial window that calls the function "is_congruent" on the factors *color* and *word*.

In [43]:
congruent = DerivedLevel('congruent', WithinTrial(is_congruent,   [color, word]))
incongruent = DerivedLevel('incongruent', WithinTrial(is_incongruent,   [color, word]))

#### Defining the congruency factor
As with the other factors, we now can define the congruency factor with the levels we just created.

In [44]:
congruency = Factor(name='congruency', initial_levels=[congruent, incongruent])

### Transition Factor: Response Transition
We also want to counterbalance (and cross) the response transitions. Again, we need to define the functions, levels, and finally the factor. The procedure is the same as with derived factors, but instead of a WithinTrial window, we will use a transition Window. In the functions, we will index the input arguments to indicate what trial to use (0 is current -1 is previous, ...)

In [45]:
## FUNCTIONS
# A trial is a response repetition trial if the current color is not the same as the previous color
def is_response_repetition(col):
    return col[0] == col[-1]

# A trial is a response switch trial if it is not a response repeatition trial
def is_response_switch(col):
    return not is_response_repetition(col)

## LEVELS
response_repetition = DerivedLevel('response_repetition', Transition(is_response_repetition, [color]))
response_switch = DerivedLevel('response_switch', Transition(is_response_switch, [color]))

## FACTOR
response_transition = Factor(name='response_transition', initial_levels=[response_repetition, response_switch])

### Constraints
SweetPea allows multiple sequential constraints for the created task sequences. For example, how many response repetition are allowed in a row. Here, the only constraint we are using is that our sequence should at least have 20 trials. For more on this, see [sweetPea-Constraints](https://sweetpea-org.github.io/api/constraints.html).

In [46]:
minimum_trials_constraint = MinimumTrials(20)

### Define the experiment
We can now define the design (what factors to include) and the crossing of the experiment. As mentioned before, sweetPea also allows for multiple individual crossings, but here we will use a single crossing.

In [47]:
# Define the design, crossing and constraints
design = [color, word, congruency, response_transition]
crossing = [color, word, response_transition]
constraints = [minimum_trials_constraint]

# Create a crossed block
block = CrossBlock(design=design, crossing=crossing, constraints=constraints)

### Synthesize sequences
With the block defined, we can now synthesize trial sequences. Let's create 5 sequences and look at them. Here, we also pass in an argument called sampling_strategy. This indicates what strategy to use. Strategies differ in *speed* for creating trial sequences, *uniformity* of the sequence distribution, and *replacement* (can a trial sequence occur multiple times). We suggest using `CMSGen` in most cases. For more on this, see [sweetPea-Sampling Strategies](https://sweetpea-org.github.io/api/sampling_strategies.html)

In [48]:
experiment = synthesize_trials(block=block, samples=5, sampling_strategy=CMSGen)

Sampling 5 trial sequences using CMSGen.
 'response_transition' depends on 'color'
Encoding experiment constraints...
Running CMSGen...


In [49]:
# now let's look at the sequences
print_experiments(block, experiment)


5 trial sequences found.

Experiment 0:
color red  | word RED  | response_transition    | congruency congruent                   
color blue | word BLUE | response_transition response_switch | congruency congruent                   
color blue | word BLUE | response_transition response_repetition | congruency congruent                   
color blue | word RED  | response_transition response_repetition | congruency incongruent                 
color red  | word BLUE | response_transition response_switch | congruency incongruent                 
color blue | word RED  | response_transition response_switch | congruency incongruent                 
color red  | word RED  | response_transition response_switch | congruency congruent                   
color blue | word RED  | response_transition response_switch | congruency incongruent                 
color red  | word BLUE | response_transition response_switch | congruency incongruent                 
color red  | word BLUE | response_tra

### Export for jsPsych
The trial sequences need to be in specific format to use in our online experiment. Luckily, sweetPea provides a convenience function to convert the sequence into this format. Let's convert the sequences and store them into a file. Here, we create a javascript file with a single object containing the trial sequences.

In [55]:
# jsPsych expects feature sequences as dicts
as_dicts = experiments_to_dicts(block, experiment)

# store the sequence list into
import json
with open('trialSequences.js', 'w') as f:
    f.write('const trialSequences = ')
    json.dump(as_dicts, f)

Copy or download the file. In the Tutorial